In [1]:
# !git clone https://github.com/fadhleryani/malti_arabi_fst.git
# # !git pull

# %pip install pynini
# %pip install pyfoma

In [2]:
# %cd malti_arabi_fst

In [308]:
import pynini as pn
import kenlm
from itertools import product
import pyconll
import pandas as pd
import numpy as np


In [ ]:
wordmodel = kenlm.Model('aggregated_country/lm/word/tn-maghreb.arpa')
charmodel = kenlm.Model('aggregated_country/lm/char/tn-maghreb.arpa')

In [293]:
mudt = pyconll.load_from_file('mt_mudt-ud-dev.conllu')

keys = ["id","form","lemma","upos","xpos","feats","head","deprel","deps","misc"]

sents = []
for sent in mudt:
    toks = [pd.Series({'sent_id':sent.id,'sent':sent.text})]
    # toks = []
    for tok in sent:
        tokdict = {'sent_id':sent.id}
        tokdict.update( {k:tok.__getattribute__(k) for k in keys})
        toks.append (pd.Series(tokdict))

    sents.append(pd.DataFrame(toks))

df = pd.concat(sents)

# word_hist
word_hist = df['sent'].dropna().str.split().explode().value_counts()
# word_hist.to_clipboard()

# char hist
char_hist = pd.DataFrame([y for x in df['sent'].dropna().str.split().explode().str.casefold().unique() for y in x]).value_counts()
# char_hist.to_clipboard()


In [322]:
words_df= pd.read_fwf('tn-maghreb-words.txt',header=None).rename(columns={0:'words'})
def dediac_fst(text):
    try:
        return (text @ dediac).string()
    except:
        return np.nan
words_df['dediac'] = words_df['words'].apply(dediac_fst)

words_df

,words,dediac
0,يا,يا
1,ديني,ديني
2,محلالي,محلالي
3,فرحو,فرحو
4,❤,NaN
...,...,...
203089,يهبلوا,يهبلوا
203090,لعمرباش,لعمرباش
203091,فالفوضى,فالفوضى
203092,خزانتك,خزانتك


In [377]:
closedclass = pd.read_csv('closed_class_mappings.tsv',sep='\t',header=None) # already unique 
closedclass = dict(closedclass.values)


In [596]:
malti2arabi = pn.string_file('malti2arabi.map')

malti_sigma = pn.project(malti2arabi,'input')
arabi_sigma = pn.project(malti2arabi,'output') 

sigma_input = pn.union(malti_sigma ).closure()

transcriber = pn.union(malti2arabi ).closure()

dediac_cross = pn.string_file('dediac.map')

diacs = 'ًٌٍَُِّْ'
dediac = pn.cdrewrite(dediac_cross,'','',pn.union(arabi_sigma,*diacs).closure())

words = pn.string_file('tn-maghreb-words.txt').optimize() @ dediac
closedclass_fst = pn.string_file('closed_class_mappings.tsv').optimize()

In [597]:
def get_paths(fst,words_only=False):
    paths = list(fst.paths().items())
    if words_only:
        return [x[1] for x in paths]
    else:
        return paths
    
def translit_fst(tok):    
    fst = (f'<BOS>{tok}<EOS>')  @ transcriber  
    # fst = (f'<BOS>{tok}<EOS>')  @ transcriber @ dediac 
    return fst

def remove_plus(text):
    return text.replace('+ ','')
    # return (text @ pn.cdrewrite(pn.cross('+'+pn.accep(' ').ques,''),'','',arabi_sigma.closure(),direction='rtl')).string() # takes a long time

def filter_edge_diacritics(options):
    return [y for y in options if y[0] not in diacs and y[-1] not in diacs]


def translit_closed_class(word):
    outvalue = []
    lowered = word.lower()
    for tok in lowered.split(): 
        outvalue.append(closedclass.get(tok,'#na'))

    return ' '.join(outvalue)

def translit_word(lowered,cutoff=3,useclosedclass=False): #select on merged but return unmerged
    lattice = []
     # iterate on lower cased
    for tok in lowered.replace('-','- ').replace("'","' ").split():
        if useclosedclass :
            cc = closedclass.get(tok)
            if cc:
                lattice.append([cc])
                continue
        tok_fst = translit_fst(tok)
        translit_toks = get_paths(tok_fst,words_only=True) 
        if not translit_toks:
            translit_toks = [""]
            continue
        try:
            translit_toks = filter_edge_diacritics(translit_toks) 
        except:
            print('err',translit_toks,tok,lowered)
        translit_toks = [(x @ dediac).string() for x in translit_toks]  # dediacritize
        lattice.append(translit_toks)
    
    return [' '.join(x) for x in product(*lattice)]

langmodelset =  set(words_df['dediac'])
def is_in_lang_model(text):
    matches = []
    for tok in text.split():
        matches.append(tok in langmodelset)
    return all(matches)



def translit_and_rank_options(word,useclosedclass=False,cutoff=None):
    lowered = word.lower()
    translit = translit_word(lowered,useclosedclass=useclosedclass)
    if not translit:
        translit = [['']]
    df = pd.DataFrame({'original_word':word,'translit':translit})
    df['word'] = lowered.replace('-','- ').replace("'","' ")
    df['merged'] = df['translit'].str.replace('+ ','')
    df['wordmodel_score'] = df['merged'].apply(wordmodel.score) 
    df['charmodel_score'] = df['merged'].apply(lambda x: charmodel.score(' '.join(x))) 
    df['capitalized'] = word[0].isupper() # TODO: what about letter after sink as in 'L-Innu', does it matter?
    df['in_langmodel'] = df['merged'].apply(is_in_lang_model) # if all tokens are in lang model
    df['closed_class'] = df['word'].apply(translit_closed_class)
    # df['translit_closed_class'] = [translit_cc]*len(df)
    if useclosedclass:
        df.rename(columns={'translit':'translit_cc'},inplace=True)
    # df['fertility'] = df['word'].apply(get_fertility) #TODO
    if cutoff:
        return df.drop_duplicates('merged').sort_values('charmodel_score',ascending=False)[:cutoff]    
    return df.drop_duplicates('merged').sort_values('charmodel_score',ascending=False)
    return df.drop_duplicates('merged').sort_values('wordmodel_score',ascending=False)


word = "t'"
word = "uffiċjali"
word = "L-Innu"
word = "din"
word = "f'dik"
word = "d-dinja"
word = "f'dil-konferenza d-dinja"
word = "d-dinja"
word = "m'għandhomx" 
word = "mil-dinja" # check how many tokens it breaks into and how that affects lang model scores
translit_and_rank_options(word,cutoff=None,useclosedclass=False).merge(translit_and_rank_options(word,cutoff=None,useclosedclass=True),how='outer').sort_values(['charmodel_score'],ascending=False)
translit_and_rank_options(word,cutoff=None,useclosedclass=False).merge(translit_and_rank_options(word,cutoff=None,useclosedclass=True),how='outer').sort_values(['wordmodel_score'],ascending=False)
# sorted([(wordmodel.score(x),x) for x in merged],key=lambda x: -x[0])


,original_word,translit,word,merged,wordmodel_score,charmodel_score,capitalized,in_langmodel,closed_class,translit_cc
96,mil-dinja,NaN,mil- dinja,من الدنيا,-6.381810,-10.657368,False,True,من ال+ #na,من ال+ دنيا
0,mil-dinja,مال+ دنيا,mil- dinja,مالدنيا,-7.383401,-7.550520,False,True,من ال+ #na,NaN
61,mil-dinja,ميال+ ضنيا,mil- dinja,ميالضنيا,-8.091814,-14.353223,False,False,من ال+ #na,NaN
71,mil-dinja,ميل+ ضنيى,mil- dinja,ميلضنيى,-8.091814,-15.061767,False,False,من ال+ #na,NaN
70,mil-dinja,ميل+ دينيى,mil- dinja,ميلدينيى,-8.091814,-14.968325,False,False,من ال+ #na,NaN
...,...,...,...,...,...,...,...,...,...,...
104,mil-dinja,NaN,mil- dinja,من الذنية,-9.945451,-14.147204,False,False,من ال+ #na,من ال+ ذنية
103,mil-dinja,NaN,mil- dinja,من الضنيا,-9.945451,-13.943551,False,False,من ال+ #na,من ال+ ضنيا
102,mil-dinja,NaN,mil- dinja,من الضنيه,-9.945451,-13.713515,False,False,من ال+ #na,من ال+ ضنيه
101,mil-dinja,NaN,mil- dinja,من الضنية,-9.945451,-13.518830,False,False,من ال+ #na,من ال+ ضنية


In [ ]:
# tuple(original_word,
# boolean(capitalized or not),
# closed class mapping or nill    ,
# boolean(Exists or not),
# wordmodel score,
# charactermodel score,
# kurts fertility)

# ADD
# freq counts (including capitalized and ignoring)
# merge and uniq all sets

# do example with dik id-dinja / fil- dinja
# think about redoing closed class to avoid changing number of tokens

# keep track if length of input is same as output (number of tokens)

# assume tokenized data (like mudt tokens)?

In [423]:
def translit_sent(sent):
    lattice = []
    for word in sent.split():
        optionsdf = translit_and_rank_options(word)
        options = optionsdf['merged'].values
        lattice.append(options)

    return list([' '.join(x) for x in product(*lattice)])
        
def score_generated_sentences(sentences):
    return sorted([(x,model.score(x)) for x in sentences],key=lambda y: -y[1])


# score_generated_sentences(translit_sentence('malta magħrufa uffiċjalment bħala',cutoff=1))

sentout = translit_sent("kien bilqiegħda f'dik il-parti")

In [424]:
dfsentout = pd.DataFrame({'sent':sentout})
dfsentout['score'] = dfsentout['sent'].apply(wordmodel.score)
dfsentout.sort_values('score',ascending=False)

,sent,score
0,كان بلقعدة في ديك البارتي,-23.318352
3,كان بلقاعدة في ديك البارتي,-23.318352
6,كان بلقعده في ديك البارتي,-23.318352
1,كان بلقعدة في ديك البرتي,-23.817825
4,كان بلقاعدة في ديك البرتي,-23.817825
7,كان بلقعده في ديك البرتي,-23.817825
2,كان بلقعدة في ديك البرطي,-23.861692
5,كان بلقاعدة في ديك البرطي,-23.861692
8,كان بلقعده في ديك البرطي,-23.861692


In [503]:
word_hist = word_hist
mudtdev_translit = pd.concat(word_hist['sent'].iloc[:].apply(translit_and_rank_options).values)


err ['فع', '']
err ['بع', '']
err ['فع', '']
err ['فع', '']
err ['مع', '']
err ['فع', '']
err ['بع', '']
err ['بع', '']
err ['فع', '']
err ['بع', '']
err ['بع', '']
err ['فع', '']
err ['سع', 'صع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['سع', 'صع', '']
err ['فع', '']
err ['مع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['بع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['فع', '']
err ['مع', '']
err ['فع', '']
err ['بع', '']
err ['فع', '']


KeyboardInterrupt: 

In [500]:
mudtdev_translit

15.1875

In [459]:
word_hist['sent'].iloc[18]

'"'

In [472]:
translit_and_rank_options('"')

[]

In [535]:
import numpy as np
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-mix")


def get_number_of_tokens(token_candidates, tokenizer):
    if len(token_candidates) == 0:
        print("No candidates given!")
        return None

    encoded = tokenizer(token_candidates, add_special_tokens=False, return_length=True)
    return encoded['len']
    token_candidates = np.array(token_candidates)

    # np.argmin(encoded["length"])
    fewest_splits = min(encoded["length"])
    shortest = list(set(token_candidates[np.where(encoded["length"] == min(encoded["length"]))]))
    if len(shortest) > 1:
        print(f"Choosing a candidate randomly from {shortest}")
    return shortest[0]



In [536]:

token_candidates = get_paths(translit_fst("malta"))
token_candidates = list(map(lambda x: x[1], token_candidates))
# token_candidates

In [539]:
list(zip(token_candidates,get_number_of_tokens(token_candidates,tokenizer)['length']))

[('مالتا', 2),
 ('مالتَ', 1),
 ('مالتا', 2),
 ('مالتة', 2),
 ('مالته', 2),
 ('مالتى', 2),
 ('مالطا', 1),
 ('مالطَ', 1),
 ('مالطا', 1),
 ('مالطة', 2),
 ('مالطه', 2),
 ('مالطى', 2),
 ('مَلتا', 1),
 ('مَلتَ', 1),
 ('مَلتا', 1),
 ('مَلتة', 1),
 ('مَلته', 1),
 ('مَلتى', 1),
 ('مَلطا', 1),
 ('مَلطَ', 1),
 ('مَلطا', 1),
 ('مَلطة', 1),
 ('مَلطه', 1),
 ('مَلطى', 1)]

In [534]:
token_candidates

['مالتا',
 'مالتَ',
 'مالتا',
 'مالتة',
 'مالته',
 'مالتى',
 'مالطا',
 'مالطَ',
 'مالطا',
 'مالطة',
 'مالطه',
 'مالطى',
 'مَلتا',
 'مَلتَ',
 'مَلتا',
 'مَلتة',
 'مَلته',
 'مَلتى',
 'مَلطا',
 'مَلطَ',
 'مَلطا',
 'مَلطة',
 'مَلطه',
 'مَلطى']